## Building a Regression Model using Keras

### Table of contents
1. Donwnload and clean dataset
2. Import Keras
3. Build a Neural Network
4. Train and Test the Neural Network

### Download and Clean Dataset

We will start by importing the necessary libraries for the project

In [1]:
import time
import os
import pandas as pd
import numpy as np


from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


In [2]:
conda install -c conda-forge keras

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


The dataset that we will use is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used. The ingredients include:
1. Cement
2. Blast furnace slag
3. Fly ash
4. Water
5. Superplasticizer
6. Coarse aggregate
7. Fine aggregate

We will download the data and read it into a pandas daataframe

In [3]:
concrete_df = pd.read_csv("/Users/juan/Downloads/concrete_data.csv")
concrete_df.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


The first sample consists of 540 cubic meters of cement, 0 cubic meters of blast furnace slag and fly ash, 162 cubic meters of water, 2.5 cubic meters of superplasticizer, 1040 cubic meters of fine aggregate, 676 cubic meters of fine aggregate and has a compressive strength of 79.99 MPa at 28 days.

Check how many data points we have in the data frame

In [4]:
concrete_df.shape

(1030, 9)

The next step is to check the dataset for any missing values

In [5]:
concrete_df.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_df.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The dataset looks clean so we can proceed to build our model

#### Split data intro predictors and target

In [7]:
concrete_df_columns = concrete_df.columns
predictors = concrete_df[concrete_df_columns[concrete_df_columns != "Strength"]] # All columns except Strength
target = concrete_df["Strength"] #Only the column Strength

Check the first rows for the predictors and target data

In [8]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [9]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Let's save the number of predictors to n_cols variables, because we will need it when we build our network

In [10]:
n_cols = predictors.shape[1] #number of predictors

### Importing packages from Keras library

Let's import the packages from the Keras library that we will need to build our regression model

In [11]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


### Part A: Build a baseline model

Use the Keras library to build a neural network with the following:

One hidden layer of 10 nodes, and a ReLU activation function

Use the adam optimizer and the mean squared error as the loss function.

Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the train_test_split helper function from Scikit-learn.

Train the model on the training data using 50 epochs.

Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

Report the mean and the standard deviation of the mean squared errors.

#### Build the neural network model

Let's create a function that defines our regression model and we can call it to create our model

In [12]:
#define the regression model with one hidden layer
def regression_model():
    # Create model
    model = Sequential()

    model.add(Dense(10, activation="relu", input_shape=(n_cols,)))
    model.add(Dense(1))

    # Compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

####  Train and test the model

We will create the regression model

In [13]:
model = regression_model()

Instructions for updating:
Colocations handled automatically by placer.


Randomly split the data into a training and test sets by holding 30%  of the data for testing. You can use the train_test_split helper function 

In [14]:
X = predictors
y = target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=24)   
print("Training set: ", X_train.shape, y_train.shape)
print("Testing set: ", X_test.shape, y_test.shape)

Training set:  (721, 8) (721,)
Testing set:  (309, 8) (309,)


Train the model on the training data using 50 epochs

In [15]:
model.fit(X_train, y_train, epochs=50, verbose=2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
 - 0s - loss: 147687.0670
Epoch 2/50
 - 0s - loss: 88079.6639
Epoch 3/50
 - 0s - loss: 51429.6772
Epoch 4/50
 - 0s - loss: 25998.9317
Epoch 5/50
 - 0s - loss: 9984.8584
Epoch 6/50
 - 0s - loss: 3074.6764
Epoch 7/50
 - 0s - loss: 1320.3720
Epoch 8/50
 - 0s - loss: 1078.8339
Epoch 9/50
 - 0s - loss: 1031.8237
Epoch 10/50
 - 0s - loss: 987.4230
Epoch 11/50
 - 0s - loss: 942.8682
Epoch 12/50
 - 0s - loss: 902.3165
Epoch 13/50
 - 0s - loss: 860.6361
Epoch 14/50
 - 0s - loss: 822.1550
Epoch 15/50
 - 0s - loss: 785.2765
Epoch 16/50
 - 0s - loss: 749.7746
Epoch 17/50
 - 0s - loss: 717.2100
Epoch 18/50
 - 0s - loss: 687.6735
Epoch 19/50
 - 0s - loss: 656.7912
Epoch 20/50
 - 0s - loss: 628.9895
Epoch 21/50
 - 0s - loss: 602.8013
Epoch 22/50
 - 0s - loss: 578.7804
Epoch 23/50
 - 0s - loss: 555.5392
Epoch 24/50
 - 0s - loss: 534.8057
Epoch 25/50
 - 0s - loss: 515.3516
Epoch 26/50
 - 0s - loss: 497.1543
Epoch 27/50
 - 0s - loss: 479.3742
Ep

Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.    
    

In [16]:
y_hat = model.predict(X_test)   
mse = mean_squared_error(y_test, y_hat)

In [17]:
print(mse)

284.2353282050415


Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors

We will create one function that performs steps 1 to 3 and another function that iterates 50 times and creates the list of 50 mean squared errors

In [18]:
def get_mean_squared_error(compiled_model, X, y, epochs=50, verbose=1):
    """Get report (dataframe) of two metrics: 
    The mean and the standard deviation of the mean squared errors
    """   
    
    # 1. Randomly split the data into a training and test sets by holding 30% 
    # of the data for testing. You can use the train_test_split helper function 
    # from Scikit-learn. 
    X = predictors
    y = target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=24)   
    print("Training set: ", X_train.shape, y_train.shape)
    print("Testing set: ", X_test.shape, y_test.shape)
    
    
    # 2. Train the model on the training data using 50 epochs.
    # Fit the built model with training set
    model.fit(X_train, y_train, epochs=epochs, verbose=verbose)    

    # 3. Evaluate the model on the test data and compute the mean squared error 
    # between the predicted concrete strength and the actual concrete strength. 
    # You can use the mean_squared_error function from Scikit-learn.    
    y_hat = model.predict(X_test)    
    mse = mean_squared_error(y_test, y_hat)
    
    # Return the mean squared error
    return mse

In [19]:
#Function to round the calculation of the mean and std deviation to 2 decimal places
def get_round(score, num_of_digits=2):
    """Get round with given number of decimal digits 
    """
    return round(score, num_of_digits)

#Function to calculate the mean of the list of mean squared errors
def get_mean(list_of_mse_scores):
    """Get mean
    """
    if list_of_mse_scores:
        return get_round(np.mean(list_of_mse_scores))
    return None

#Function to calculate the standard deviation of the list of mean squared errors
def get_standard_deviation(list_of_mse_scores):
    """Get standard deviation
    """
    if list_of_mse_scores:
        return get_round(np.std(list_of_mse_scores))
    return None




#Function to iterate and calculate the mean squared error
def get_mean_and_std_of_mse(df_X, 
                            df_y, 
                            compiled_model,                
                            max_iteration=50, 
                            epochs=50, 
                            verbose=0):
    """Generate the mean and the standard deviation of the mean squared errors 
    """
    # Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.    
    list_of_mean_squared_errors = []
    for i in range(max_iteration):
        start_time = time.time()
        print("-" * 36)
        print("Processing current number of iteration : {}".format(i+1))        
        mse = get_mean_squared_error(compiled_model, df_X, df_y, epochs=epochs, verbose=verbose)
        list_of_mean_squared_errors.append(mse)
        print("Duration (seconds): {}".format(time.time()-start_time))
    # end for

    print("Finished - {} times.\nAnd the list of mean squared errors : {}".format(max_iteration,
                                                                                  list_of_mean_squared_errors))
    
    mean_mse = get_mean(list_of_mean_squared_errors)
    std_mse = get_standard_deviation(list_of_mean_squared_errors)

    print("-" * 72)
    print("The mean and the standard deviation of the mean squared errors are: {} and {}, respectively".format(
           mean_mse, std_mse))
    
    return mean_mse, std_mse




In [20]:
max_iteration = 50
epochs = 50
verbose = 2

# Get the compiled model
model = regression_model()

mean_mse, std_mse = get_mean_and_std_of_mse(predictors, target, model, max_iteration=max_iteration, epochs=epochs, verbose=verbose)

------------------------------------
Processing current number of iteration : 1
Training set:  (721, 8) (721,)
Testing set:  (309, 8) (309,)
Epoch 1/50
 - 0s - loss: 3325.9508
Epoch 2/50
 - 0s - loss: 1584.1490
Epoch 3/50
 - 0s - loss: 1251.4977
Epoch 4/50
 - 0s - loss: 1053.0818
Epoch 5/50
 - 0s - loss: 895.0838
Epoch 6/50
 - 0s - loss: 768.7322
Epoch 7/50
 - 0s - loss: 666.6994
Epoch 8/50
 - 0s - loss: 590.3865
Epoch 9/50
 - 0s - loss: 523.4443
Epoch 10/50
 - 0s - loss: 468.7924
Epoch 11/50
 - 0s - loss: 426.2245
Epoch 12/50
 - 0s - loss: 391.4226
Epoch 13/50
 - 0s - loss: 362.8815
Epoch 14/50
 - 0s - loss: 337.3791
Epoch 15/50
 - 0s - loss: 313.0196
Epoch 16/50
 - 0s - loss: 296.8287
Epoch 17/50
 - 0s - loss: 273.4657
Epoch 18/50
 - 0s - loss: 263.6529
Epoch 19/50
 - 0s - loss: 245.2958
Epoch 20/50
 - 0s - loss: 232.4135
Epoch 21/50
 - 0s - loss: 219.8393
Epoch 22/50
 - 0s - loss: 210.6402
Epoch 23/50
 - 0s - loss: 200.9771
Epoch 24/50
 - 0s - loss: 189.7150
Epoch 25/50
 - 0s - loss

### Report the mean and standard deviation of the mean squared error

The mean and standard deviation of the mean squared error after 50 iterations, for the case of not normalized data is:

In [21]:
def get_report(name_of_case, mean_mse, std_mse):
    """Get report of mse and std: 
    The mean and the standard deviation of the mean squared errors
    """
    COL_NAME_EXPERIMENT = "Experiment"
    COL_NAME_MSE = "Mean MSE"
    COL_NAME_RMSE = "Std Deviation MSE"
    header_of_mse_and_rmse = [COL_NAME_EXPERIMENT, COL_NAME_MSE, COL_NAME_RMSE]
    values = [[name_of_case, mean_mse, std_mse]]

    return pd.DataFrame(columns=header_of_mse_and_rmse, data=values)

In [22]:
name_of_case = "Baseline-not normalized (50 epochs)"

# Report the mean and the standard deviation of the mean squared errors
df_baseline = get_report(name_of_case, mean_mse, std_mse)
df_baseline

,Experiment,Mean MSE,Std Deviation MSE
0,Baseline-not normalized (50 epochs),55.41,13.89


## Part B: Normalized data

Repeat Part A but use a normalized version of the data. Recall that one way to normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.
How does the mean of the mean squared errors compare to that from Step A?

### Data before normalization

In [23]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


### Data after normalization

We will normalize the data by substracting the mean and dividing by the standard deviation

In [24]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [25]:
n_colsnorm = predictors_norm.shape[1] #number of predictors

Defining the regression model as above

In [26]:
#define the regression model with one hidden layer
def regression_model():
    # Create model
    model = Sequential()

    model.add(Dense(10, activation="relu", input_shape=(n_colsnorm,)))
    model.add(Dense(1))

    # Compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

Creating the regression model

In [27]:
model = regression_model()

Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the train_test_split helper function

In [28]:
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=24)   
print("Training set: ", X_train.shape, y_train.shape)
print("Testing set: ", X_test.shape, y_test.shape)

Training set:  (721, 8) (721,)
Testing set:  (309, 8) (309,)


Train the model on the training data using 50 epochs

In [29]:
model.fit(X_train, y_train, epochs=50, verbose=2)

Epoch 1/50
 - 0s - loss: 1615.0911
Epoch 2/50
 - 0s - loss: 1601.5464
Epoch 3/50
 - 0s - loss: 1588.7095
Epoch 4/50
 - 0s - loss: 1576.3353
Epoch 5/50
 - 0s - loss: 1563.9033
Epoch 6/50
 - 0s - loss: 1551.2929
Epoch 7/50
 - 0s - loss: 1537.9528
Epoch 8/50
 - 0s - loss: 1523.9388
Epoch 9/50
 - 0s - loss: 1508.6667
Epoch 10/50
 - 0s - loss: 1492.2921
Epoch 11/50
 - 0s - loss: 1474.5051
Epoch 12/50
 - 0s - loss: 1454.9459
Epoch 13/50
 - 0s - loss: 1434.1095
Epoch 14/50
 - 0s - loss: 1410.9501
Epoch 15/50
 - 0s - loss: 1386.9014
Epoch 16/50
 - 0s - loss: 1360.5853
Epoch 17/50
 - 0s - loss: 1333.2521
Epoch 18/50
 - 0s - loss: 1304.6021
Epoch 19/50
 - 0s - loss: 1274.8266
Epoch 20/50
 - 0s - loss: 1243.6248
Epoch 21/50
 - 0s - loss: 1211.5846
Epoch 22/50
 - 0s - loss: 1178.4650
Epoch 23/50
 - 0s - loss: 1144.4540
Epoch 24/50
 - 0s - loss: 1109.9478
Epoch 25/50
 - 0s - loss: 1074.7858
Epoch 26/50
 - 0s - loss: 1039.1809
Epoch 27/50
 - 0s - loss: 1003.1913
Epoch 28/50
 - 0s - loss: 966.6377
Ep

Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

In [30]:
y_hat = model.predict(X_test)   
mse = mean_squared_error(y_test, y_hat)

In [31]:
print(mse)

296.39646635977283


For the case of not normalized data that we ran in Part A mse = 284.23, for this case with normalized data mse= 296.39

Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors

In [32]:
def get_mean_squared_error(compiled_model, X, y, epochs=50, verbose=1):
    """Get report (dataframe) of two metrics: 
    The mean and the standard deviation of the mean squared errors
    """   
    
    # 1. Randomly split the data into a training and test sets by holding 30% 
    # of the data for testing. You can use the train_test_split helper function 
    # from Scikit-learn. 
    X = predictors_norm
    y = target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=24)   
    print("Training set: ", X_train.shape, y_train.shape)
    print("Testing set: ", X_test.shape, y_test.shape)
    
    
    # 2. Train the model on the training data using 50 epochs.
    # Fit the built model with training set
    model.fit(X_train, y_train, epochs=epochs, verbose=verbose)    

    # 3. Evaluate the model on the test data and compute the mean squared error 
    # between the predicted concrete strength and the actual concrete strength. 
    # You can use the mean_squared_error function from Scikit-learn.    
    y_hat = model.predict(X_test)    
    mse = mean_squared_error(y_test, y_hat)
    
    # Return the mean squared error
    return mse

In [33]:
#Function to round the calculation of the mean and std deviation to 2 decimal places
def get_round(score, num_of_digits=2):
    """Get round with given number of decimal digits 
    """
    return round(score, num_of_digits)

#Function to calculate the mean of the list of mean squared errors
def get_mean(list_of_mse_scores):
    """Get mean
    """
    if list_of_mse_scores:
        return get_round(np.mean(list_of_mse_scores))
    return None

#Function to calculate the standard deviation of the list of mean squared errors
def get_standard_deviation(list_of_mse_scores):
    """Get standard deviation
    """
    if list_of_mse_scores:
        return get_round(np.std(list_of_mse_scores))
    return None




#Function to iterate and calculate the mean squared error
def get_mean_and_std_of_mse(df_X, 
                            df_y, 
                            compiled_model,                
                            max_iteration=50, 
                            epochs=50, 
                            verbose=0):
    """Generate the mean and the standard deviation of the mean squared errors 
    """
    # Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.    
    list_of_mean_squared_errors = []
    for i in range(max_iteration):
        start_time = time.time()
        print("-" * 36)
        print("Processing current number of iteration : {}".format(i+1))        
        mse = get_mean_squared_error(compiled_model, df_X, df_y, epochs=epochs, verbose=verbose)
        list_of_mean_squared_errors.append(mse)
        print("Duration (seconds): {}".format(time.time()-start_time))
    # end for

    print("Finished - {} times.\nAnd the list of mean squared errors : {}".format(max_iteration,
                                                                                  list_of_mean_squared_errors))
    
    mean_mse = get_mean(list_of_mean_squared_errors)
    std_mse = get_standard_deviation(list_of_mean_squared_errors)

    print("-" * 72)
    print("The mean and the standard deviation of the mean squared errors are: {} and {}, respectively".format(
           mean_mse, std_mse))
    
    return mean_mse, std_mse




In [34]:
max_iteration = 50
epochs = 50
verbose = 2

# Get the compiled model
model = regression_model()

mean_mse, std_mse = get_mean_and_std_of_mse(predictors_norm, target, model, max_iteration=max_iteration, epochs=epochs, verbose=verbose)

------------------------------------
Processing current number of iteration : 1
Training set:  (721, 8) (721,)
Testing set:  (309, 8) (309,)
Epoch 1/50
 - 0s - loss: 1574.5517
Epoch 2/50
 - 0s - loss: 1557.6679
Epoch 3/50
 - 0s - loss: 1540.8171
Epoch 4/50
 - 0s - loss: 1523.8873
Epoch 5/50
 - 0s - loss: 1506.5983
Epoch 6/50
 - 0s - loss: 1489.1528
Epoch 7/50
 - 0s - loss: 1471.0778
Epoch 8/50
 - 0s - loss: 1452.6774
Epoch 9/50
 - 0s - loss: 1433.7410
Epoch 10/50
 - 0s - loss: 1414.1822
Epoch 11/50
 - 0s - loss: 1393.3315
Epoch 12/50
 - 0s - loss: 1371.9828
Epoch 13/50
 - 0s - loss: 1348.9047
Epoch 14/50
 - 0s - loss: 1325.4215
Epoch 15/50
 - 0s - loss: 1300.1354
Epoch 16/50
 - 0s - loss: 1274.2417
Epoch 17/50
 - 0s - loss: 1247.4156
Epoch 18/50
 - 0s - loss: 1218.9390
Epoch 19/50
 - 0s - loss: 1189.5432
Epoch 20/50
 - 0s - loss: 1158.8129
Epoch 21/50
 - 0s - loss: 1126.7940
Epoch 22/50
 - 0s - loss: 1094.3861
Epoch 23/50
 - 0s - loss: 1060.9528
Epoch 24/50
 - 0s - loss: 1026.8742
Epoc

###  Report the mean and standard deviation of the mean squared error

The mean and standard deviation of the mean squared error after 50 iterations, for the case of not normalized data is:

In [35]:
def get_report(name_of_case, mean_mse, std_mse):
    """Get report of mse and std: 
    The mean and the standard deviation of the mean squared errors
    """
    COL_NAME_EXPERIMENT = "Experiment"
    COL_NAME_MSE = "Mean MSE"
    COL_NAME_RMSE = "Std Deviation MSE"
    header_of_mse_and_rmse = [COL_NAME_EXPERIMENT, COL_NAME_MSE, COL_NAME_RMSE]
    values = [[name_of_case, mean_mse, std_mse]]

    return pd.DataFrame(columns=header_of_mse_and_rmse, data=values)

In [36]:
name_of_case = "Baseline  normalized (50 epochs)"

# Report the mean and the standard deviation of the mean squared errors
df_baseline_norm = get_report(name_of_case, mean_mse, std_mse)
df_baseline_norm

,Experiment,Mean MSE,Std Deviation MSE
0,Baseline normalized (50 epochs),49.27,37.0


Comparing the case of not normalized data with the case of normalized data:

In [40]:
# Create a data frame with the summary
df_summary = pd.concat([df_baseline, df_baseline_norm], axis=0)

# Review the result dataframe
df_summary.reset_index(drop=True)


,Experiment,Mean MSE,Std Deviation MSE
0,Baseline-not normalized (50 epochs),55.41,13.89
1,Baseline normalized (50 epochs),49.27,37.00
